In [ ]:
import pandas as pd
import fitparse
from mutagen.mp4 import Atoms
from pathlib import Path
import gpxpy
import gpxpy.gpx
import calendar
import time
import struct
from datetime import datetime, timedelta

vid = R'd:\0\VIRB0569.MP4'

def read_u32(data,pos):
    return struct.unpack(">I", data[pos:pos + 4])[0]

def get_video_times(vid):
    fileobj = open(vid, "rb")
    atoms = Atoms(fileobj)
    mvhd = atoms.__getitem__(b"moov.mvhd")
    _, data = mvhd.read(fileobj)
    dt_base = datetime(1904, 1, 1, 0, 0, 0)
    video_start = read_u32(data,4)
    video_start = (dt_base + timedelta(seconds=(video_start)))
    timescale = read_u32(data,12)
    duration = read_u32(data,16)
    duration_s = duration / timescale
    video_end = (video_start + timedelta(seconds=(duration_s)))
    return video_start, video_end

def get_video_uuid(vid):
    fileobj = open(vid, "rb")
    atoms = Atoms(fileobj)
    atom = atoms.__getitem__(b"moov.udta.uuid")
    _, data = atom.read(fileobj)
    uuid = data.decode("utf-8")
    fileobj.close()
    return uuid

def msg_data(msg):
    data = {}
    for field in msg.fields:
        data[field.name] = field.value
    return data

def parse_fit_file(fit_file_path):
    fitfile = fitparse.FitFile(fit_file_path,data_processor=fitparse.StandardUnitsDataProcessor())
    camera_events = []
    gps_data = []
    for msg in fitfile.messages:
        if isinstance(msg, fitparse.DataMessage):
            match msg.name:
                case "camera_event":
                    camera_events.append(msg_data(msg))
                case "gps_metadata":
                    gps_data.append(msg_data(msg))

    return pd.DataFrame(camera_events).fillna(method='ffill'), pd.DataFrame(gps_data).fillna(method='ffill')

def dump_gpx(df:pd.DataFrame,filepath):
    gpx = gpxpy.gpx.GPX()
    gpx_track = gpxpy.gpx.GPXTrack()
    gpx.tracks.append(gpx_track)
    gpx_segment = gpxpy.gpx.GPXTrackSegment()
    gpx_track.segments.append(gpx_segment)

    for index, row in df.iterrows():
        gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(time=row['timestamp_utc_adjusted'],latitude=row['position_lat'],longitude=row['position_long'],elevation=row['enhanced_altitude'],speed=row['enhanced_speed']))

    with open(filepath, 'w') as f:
        f.write(gpx.to_xml(version="1.0"))


current_GMT = str(calendar.timegm(time.gmtime()))
uuid = get_video_uuid(vid)
print(uuid)

vid_file = Path(vid)
folder = vid_file.parent

df_camera_events = pd.DataFrame()
df_gps_data = pd.DataFrame()

fit_files = folder.glob('*.fit')
for fit in fit_files:
    df_fit_camera_events, df_fit_gps_data = parse_fit_file(str(fit.resolve()))
    df_fit_camera_events['file'] = fit.name
    df_fit_gps_data['file'] = fit.name
    
    tdelta_ms = pd.to_timedelta(df_fit_gps_data['timestamp_ms'], unit="ms")
    tdelta_s = pd.to_timedelta(df_fit_gps_data['timestamp'], unit="s")
    df_fit_gps_data['timestamp_utc'] = pd.to_datetime(df_fit_gps_data['utc_timestamp'])
    df_fit_gps_data['timestamp_utc_adjusted'] = df_fit_gps_data['timestamp_utc'].min() - tdelta_s.min() + tdelta_s
    df_fit_gps_data['timestamp_utc_adjusted'] = df_fit_gps_data['timestamp_utc_adjusted'] + tdelta_ms
    
    df_camera_events = pd.concat([df_camera_events, df_fit_camera_events])
    df_gps_data = pd.concat([df_gps_data, df_fit_gps_data])

df_gps_data.drop_duplicates(subset=['timestamp_utc_adjusted'],inplace=True)
df_gps_data = df_gps_data.loc[:,~df_gps_data.columns.str.startswith('unknown')]

df_camera_events['ts'] = df_camera_events['timestamp'] + df_camera_events['timestamp_ms']/1000
df_gps_data['ts'] = df_gps_data['timestamp'] + df_gps_data['timestamp_ms']/1000

df_gps_data['timestamp_utc_str'] = df_gps_data['timestamp_utc_adjusted'].dt.tz_localize('UTC')
df_gps_data['timestamp_utc_str'] = df_gps_data['timestamp_utc_adjusted'].dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

telemetry_cols = ['date','lat (deg)','lon (deg)','alt (m)','speed (km/h)']
df_gps_data[telemetry_cols] = df_gps_data[['timestamp_utc_str','position_lat','position_long','enhanced_altitude','enhanced_speed']]
df_gps_data['speed (m/s)'] = df_gps_data['speed (km/h)'] / 3.6
telemetry_cols.append('speed (m/s)')

gpx_cols = ['timestamp_utc_adjusted','position_lat','position_long','enhanced_altitude','enhanced_speed']

output_file = Path(f"d:\{current_GMT}")
output_gpx = output_file.with_suffix('.gpx')
output_csv = output_file.with_suffix('.csv')

dump_gpx(df_gps_data[gpx_cols], output_gpx)
print(f'Dumped at {output_gpx.resolve()}')

df_gps_data[telemetry_cols].to_csv(output_csv, index=False)
print(f'Dumped at {output_csv.resolve()}')

df_gps_data.to_csv(output_file.with_name(f'{current_GMT}_all').with_suffix('.csv'), index=False)
print(f'Dumped all data')

vid_events = df_camera_events.query('camera_file_uuid == @uuid and camera_event_type in ("video_start","video_end")')

if len(vid_events) == 2:
    
    video_start = vid_events.loc[vid_events['camera_event_type']=="video_start"]
    video_end = vid_events.loc[vid_events['camera_event_type']=="video_end"]
    video_gsp_data = df_gps_data.query(' file == @video_start["file"].iat[0] and ts >= @video_start["ts"].iat[0] and ts <= @video_end["ts"].iat[0] ')
    
    output_gpx = output_gpx.with_name(vid_file.name).with_suffix('.gpx')
    dump_gpx(video_gsp_data[gpx_cols], output_gpx)
    print(f'Dumped at {output_gpx.resolve()}')
    
    output_csv = output_csv.with_name(vid_file.name).with_suffix('.csv')
    video_gsp_data[telemetry_cols].to_csv(output_csv, index=False)
    print(f'Dumped at {output_csv.resolve()}')
    
else:
    
    video_start, video_end = get_video_times(vid)
    print("Couldn't locate gps data for particular mp4, here are the video start/end times to find them manually:")
    print(video_start.strftime('%Y-%m-%d %H:%M:%S'), video_end.strftime('%Y-%m-%d %H:%M:%S'))
    